In [13]:
import numba
from numba import jit
import numpy as np
import math
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numpy import sqrt
from numpy import abs

In [9]:
def plot_approximate_vs_exact(
    approximate,
    exact_fun,
    legend_loc="upper left",
    exact_plot_div=100
):
    X, Y = approximate
    plt.scatter(X, Y, label="approximate")

    EX_for_graph = np.linspace(X[0], X[-1], exact_plot_div)
    EY_for_graph = exact_fun(EX_for_graph)

    EY = exact_fun(X)
    rmse = math.sqrt(mean_squared_error(Y, EY))
    error = abs(Y[-1] - EY[-1])
    print("last point error =", error)
    print("rmse =", rmse)
    
    plt.plot(EX_for_graph, EY_for_graph, label="exact")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend(loc=legend_loc)
    plt.show()

In [68]:
@jit(forceobj=True)
def rkacem_system_method(T0, X0, h, getFunc, tol):
    """
        returns (X1, K1) where X1 is value at T0+h, and K1 is slop at T0.
        K1 can be used to controll algorithm iteration
    """
    size = X0.shape[0]
    while True:
        # Todo perform the loop operation in more efficient way
        K1 = np.zeros(size)
        for j in range(0, size):
            K1[j] = getFunc(j)(T0, X0)

        K2 = np.zeros(size)
        for j in range(0, size):
            K2[j] =  getFunc(j)(T0 + h/2, X0 + h*K1/2)

        K3 = np.zeros(size)
        for j in range(0, size):
            K3[j] = getFunc(j)(T0 + h/2, X0 + h*K2/2)

        K4 = np.zeros(size)
        for j in range(0, size):
            K4[j] = getFunc(j)(T0 + h, X0 + h*K3)
        XKh = X0 + h*(K1 + 2*K2+ 2*K3 + K4)/6

        SK1 = K1
        SK2 = K2

        SK3 = np.zeros(size)
        for j in range(0, size):
            SK3[j] = getFunc(j)(T0 + h/2, X0 + h*SK1/24 + (h*11*SK2)/24)

        SK4 = np.zeros(size)
        for j in range(0, size):
            SK4[j] = getFunc(j)(T0 + h, X0 + h*SK1/12 - (h*25*SK2)/132 + (h*73*SK3/66))

        XSh = X0 + ((2*h)/9) * ((SK1*SK1+SK1*SK2+SK2*SK2)/(SK1+SK2) + (SK2*SK2+SK2*SK3+SK3*SK3)/(SK2+SK3) + (SK3*SK3+SK3*SK4+SK4*SK4)/(SK3+SK4))

        errest = abs(XKh - XSh)* (281/13824)
        # print(errest)
        delta = .84 * ((tol/errest)**0.25)
        
        #Todo try to make a algorithm without the below line
        min_delta = np.min(delta)
        max_errest = np.max(errest)
        
        h_next = min_delta * h
        if max_errest < tol:
            break
        else:
            h = h_next
            
    return (XKh, h, h_next)

@jit(forceobj=True)
def rkhem_system_method(T0, X0, h, getFunc, tol):
    """
        returns (X1, K1) where X1 is value at T0+h, and K1 is slop at T0.
        K1 can be used to controll algorithm iteration
    """

    size = X0.shape[0]
    while True:
        # Todo perform the loop operation in more efficient way
        
        K1 = np.zeros(size)
        for j in range(0, size):
            K1[j] =  getFunc(j)(T0, X0)

        K2 = np.zeros(size)
        for j in range(0, size):
            K2[j] = getFunc(j)(T0 + h/2, X0 + h*K1/2)

        K3 = np.zeros(size)
        for j in range(0, size):
            K3[j] =  getFunc(j)(T0 + h/2, X0 + h*K2/2)

        K4 = np.zeros(size)
        for j in range(0, size):
            K4[j] =  getFunc(j)(T0 + h, X0 + h*K3)
        XKh = X0 + h*(K1 + 2*K2+ 2*K3 + K4)/6

        SK1 = K1
        SK2 = K2

        SK3 = np.zeros(size)
        for j in range(0, size):
            SK3[j] =  getFunc(j)(T0 + h/2, X0 - h*SK1/44 + (h*25*SK2)/48)

        SK4 = np.zeros(size)
        for j in range(0, size):
            SK4[j] = getFunc(j)(T0 + h, X0 - h*SK1/24 + (h*47*SK2)/600 + (h*289*SK3/300))

        XSh = X0 + (h/9) * (SK1 + 2*(SK2 + SK3) + SK4 + sqrt(abs(SK1*SK2)) + sqrt(abs(SK2*SK3)) + sqrt(abs(SK3*SK4)))

        errest = abs(XKh - XSh)* (121809/1658880)
        # print(errest)
        delta = .84 * ((tol/errest)**0.25)
        
        #Todo try to make a algorithm without the below line
        min_delta = np.min(delta)
        max_errest = np.max(errest)
        
        h_next = min_delta * h
        if max_errest < tol:
            break
        else:
            h = h_next
            
    return (XKh, h, h_next)

# ode_system_algorithm
@jit(forceobj=True)
def rk4_system_algorithm(t0, X0, tn, h, getFunc, ode_method, tol=100, max_tolerable_dynamic=-1):
    """
        stop algorithm iteration when max slop is less than stop_dynamic
        and stop at tn.
    """
    size = X0.shape[0]
    T = np.array([t0])
    XX = np.array([X0])
    
    i = 0
    max_abs_dynamic = math.inf
    while (T[i] < tn):
        i = i + 1

        Xh, h_prev, h = ode_method(T[i-1], XX[i-1], h, getFunc, tol)
        T = np.append(T, T[i-1]+h_prev)
        XX = np.append(XX, [Xh], axis=0)
        
        # return when slop is close to zero because if the slop is close to zero then 
        # system will not advance. so the current time value can be considered as final
        # time value.
        K = np.zeros(size)
        for j in range(0, size):
            K[j] = getFunc(j)(T[i], XX[i])
        max_dynamic = np.max(abs(K))
        if max_dynamic < max_tolerable_dynamic:
            return (T, XX)
        
    h = tn - T[i-1]
    Xh, h_prev, h = ode_method(T[i-1], XX[i-1], h, getFunc, tol)
    XX[i] = Xh
    T[i] = T[i-1] + h_prev

    return (T, XX)

@jit(forceobj=True)
def test():

    @jit(forceobj=True, nopython = True)
    def f1(t, X:np.array):
        return X[0] - X[1] + 2
    
    @jit(forceobj=True, nopython = True)
    def f2(t, X:np.array):
        return -X[0] + X[1] + 4*t
        
    @jit(forceobj=True, nopython = True)
    def getFunc(index):
        if (index == 0):
            return f1
            # return lambda t, X: X[0] - X[1] + 2
        if (index == 1):
            return f2
            # return lambda t, X: -X[0] + X[1] + 4*t

    step = .01
    tol = .0001
    
    # T, XX = rk4_system_algorithm(0, np.array([-1, 0]), 1.5, step, getFunc, rkhem_system_method, tol=tol)    

# X = XX[:, 0]
# Y = XX[:, 1]

# plot_approximate_vs_exact((T, X), lambda x : -0.5*np.exp(2*x) + x**2 + 2*x - 0.5)
# plot_approximate_vs_exact((T, Y), lambda x : 0.5*np.exp(2*x) + x**2 - 0.5)

%timeit test()

TypingError: Failed in object mode pipeline (step: convert make_function into JIT functions)
[1mCannot capture the non-constant value associated with variable 'f1' in a function that will escape.
[1m
File "..\..\..\..\..\AppData\Local\Temp\ipykernel_16136\2357248818.py", line 162:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [64]:
@jit(forceobj=True)
def test():
    @jit(forceobj=True)
    def getFunc(index):
        if (index == 0):
            return lambda t, X: X[0] - X[1] + 2
        if (index == 1):
            return lambda t, X: -X[0] + X[1] + 4*t

    step = .01
    tol = .0001



In [65]:
test()